In [1]:
import pandas as pd
import gtfstk as gt
import datetime as dt
import numpy as np
import tkinter as tk
import os
from tkinter import filedialog
%matplotlib inline

In [2]:
def button_gtfs_clicked():
    root.gtfs=filedialog.askopenfilename(initialdir="/",title="Select GTFS Zip file",filetypes = (("zip files","*.zip"),("all files","*.*")))
def button_outfolder_clicked():
    root.outfolder=filedialog.askdirectory(initialdir="/",title="Select Output folder")
def close_window():
    root.stop_id=entry_stop_id.get()
    root.weekday=weekday.get()
    root.destroy()


content=tk.StringVar

root=tk.Tk()
root.title('Departure Board')
# root.geometry('350x200')

label_gtfs=tk.Label(root,text="Select GTFS zip file.")
label_gtfs.grid(column=0,row=0)
button_gtfs=tk.Button(root,text="Browse",command=button_gtfs_clicked)
button_gtfs.grid(column=0,row=1)

label_out=tk.Label(root,text="Select an output folder")
label_out.grid(column=0,row=2)
button_out=tk.Button(root,text="Browse",command=button_outfolder_clicked)
button_out.grid(column=0,row=3)

label_stop_id=tk.Label(root,text="Enter Stop ID")
label_stop_id.grid(column=0,row=4)
entry_stop_id=tk.Entry(root)
entry_stop_id.grid(column=0,row=5)

label_weekday=tk.Label(root,text="Choose Weekday Type")
label_weekday.grid(column=0,row=6)
weekday=tk.StringVar()
tk.Radiobutton(root,text="Weekday",variable=weekday,value="Weekday").grid(column=0,row=7)
tk.Radiobutton(root,text="Saturday",variable=weekday,value="Saturday").grid(column=0,row=8)

button_run=tk.Button(root,text='Run',width=25,command=close_window)
button_run.grid(column=0,row=9)
root.mainloop()

In [3]:
feed=gt.read_gtfs(root.gtfs,dist_units='mi')
feed.stop_times.sort_values(['trip_id','stop_sequence'],inplace=True)
feed.stop_times['departure_time'].fillna(method='ffill',inplace=True)

In [4]:
outfolder=root.outfolder
if os.path.exists(outfolder):
    outfolder=outfolder+"\{}.xlsx"
else:
    os.mkdir(root.outfolder)
    outfolder=outfolder+"\{}.xlsx"
filename=os.path.basename(root.gtfs)
filename=os.path.splitext(filename)[0]

In [5]:
if root.weekday=='Weekday':
    savename=root.stop_id+"_"+filename+"_WeekdayDepartureBoard"
if root.weekday=='Saturday':
    savename=root.stop_id+"_"+filename+"_SaturdayDepartureBoard"

In [6]:
service_dates=gt.calendar.get_dates(feed)
service_dates=pd.DataFrame(service_dates,columns=['DateStr'])
service_dates['Date']=pd.to_datetime(service_dates['DateStr'])
service_dates['Weekday']=service_dates['Date'].dt.dayofweek
if root.weekday=='Weekday':
    service_dates=service_dates.loc[(service_dates['Weekday']>=0)&(service_dates['Weekday']<5)]
if root.weekday=='Saturday':
    service_dates=service_dates.loc[service_dates['Weekday']==5]
service_dates=service_dates['DateStr'].tolist()
dayofweek=gt.trips.compute_busiest_date(feed,service_dates)

In [7]:
def getDepartures():

    start_time=gt.stop_times.get_start_and_end_times(feed,date=dayofweek)[0]
    end_time=gt.stop_times.get_start_and_end_times(feed,date=dayofweek)[1]
    
    start_time=dt.datetime.strptime(start_time,'%H:%M:%S')
    start_time=dt.datetime(start_time.year,start_time.month,start_time.day,start_time.hour,5*(start_time.minute//5))
    start_time=start_time.strftime('%H:%M:%S')
    
    end_time=dt.datetime.strptime(end_time,'%H:%M:%S')
    end_time=dt.datetime(end_time.year,end_time.month,end_time.day,end_time.hour,5*(end_time.minute//5))
    end_time=end_time.strftime('%H:%M:%S')
    
    if end_time>='24:00:00':
        end_time='23:55:00'
        start_time='00:00:00'
    
    departures_table=gt.stops.build_stop_timetable(feed,root.stop_id,dates=dayofweek)
    departures_table=departures_table.merge(feed.routes,how='left')
    if len(departures_table['direction_id'].unique())>1:
        departures_table=departures_table.loc[departures_table['direction_id']==0]
    departures_table['RouteName']=departures_table['route_short_name']+" "+departures_table['route_long_name']
    departures_table=departures_table[['route_short_name','RouteName','route_color','route_text_color','departure_time','stop_id']]
    departures_table['route_color']="background-color:#"+departures_table['route_color']
    departures_table['route_text_color']="color:#"+departures_table['route_text_color']
    departures_table_pastMidnight=departures_table.loc[departures_table['departure_time']>='24:00:00']
    departures_table.departure_time[departures_table['departure_time'].isin(departures_table_pastMidnight['departure_time'])]=departures_table['departure_time']-pd.Timedelta(hours=24)
    departures_table['dt']=pd.to_datetime(departures_table['departure_time'])
    departures_table['dt']=departures_table['dt'].apply(lambda x: dt.datetime(x.year,x.month,x.day,x.hour,5*(x.minute//5)))
    departures_table=departures_table.pivot_table(index='dt',columns=['RouteName','route_color','route_text_color'],values='route_short_name',aggfunc='first').reset_index()
    departures_table.set_index('dt',inplace=True)
    
    column_colors=departures_table.columns.get_level_values('route_color').tolist()
    
    departures_table.columns=departures_table.columns.get_level_values('RouteName')
    
    time=pd.date_range(start=start_time,end=end_time,freq='5min')
    time=pd.DataFrame(time,columns=['dt'])
    time=time.merge(departures_table,how='left',on='dt')
    time['dt']=time['dt'].dt.strftime('%I'':''%M' ' ' '%p').apply(lambda x: x.lstrip("0"))
    time.rename({'dt':'Departure Time'},axis=1,inplace=True)
    time=time.replace(np.nan,'',regex=True)
    time.set_index('Departure Time',inplace=True)
    
    d= dict(zip(time.columns,[i for i in column_colors]))
    def mycolor(x):
        s=pd.DataFrame(d,index=x.index,columns=x.columns)
        df1=x.mask(x.replace('',np.nan).notna(),s)
        return df1
    time=time.style.apply(mycolor,axis=None)
    time.to_excel(outfolder.format(savename))

    return time
getDepartures()

,2 Broadway,3 Vista
Departure Time,,
7:20 AM,,
7:25 AM,,
7:30 AM,,
7:35 AM,,
7:40 AM,,
7:45 AM,,
7:50 AM,,
7:55 AM,,
8:00 AM,,
